In [2]:
from sympy import sin, cos, symbols, lambdify
import numpy as np
x = symbols('x')
expr = sin(x) + cos(x)
expr
f = lambdify(x, expr, 'numpy')
a = np.array([1, 2])
f(a)
# type(f(a))

array([1.38177329, 0.49315059])

In [1]:
from gymnax.environments import environment, spaces
import numpy as np
import jax
import chex
action_space = spaces.Box(-1.0,1.0,shape=(2,3),dtype=np.float32)
rng = jax.random.PRNGKey(0)
action_space.sample(rng=rng)


/home/bianzx/miniconda3/envs/pytorch3d/lib/python3.9/site-packages/flax/core/frozen_dict.py:169: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(


Array([[ 0.14900088, -0.80062795,  0.48393178],
       [ 0.78835654,  0.19313312, -0.09349632]], dtype=float32)

In [7]:

from quadjax.dynamics.dataclass import Action
import numpy as np
thrust = 1.0
tau = 0.5
env_action = Action(thrust=np.array([thrust, thrust]), tau = np.array([tau, tau]))
env_action


Action(thrust=array([1., 1.]), tau=array([0.5, 0.5]))

In [ ]:
import sympy as sp
from sympy.physics.mechanics import (
    dynamicsymbols,
    ReferenceFrame,
    Point,
    Particle,
    RigidBody,
    inertia,
)

# Define symbols
t = sp.Symbol("t")  # time
m = sp.Symbol("m", positive=True)  # mass of the quadrotor
I = sp.Symbol("I", positive=True)  # moment of inertia
g = sp.Symbol("g", positive=True)  # gravitational acceleration
l = sp.Symbol("l", positive=True)  # length of the rod
# mass of the object attached to the rod
mo = sp.Symbol("mo", positive=True)
thrust = sp.Function("thrust")(t)  # thrust force
tau = sp.Function("tau")(t)  # torque
f_rope = sp.Symbol("f_rope")  # force in the rope
# y displacement of the hook from the quadrotor center
delta_yh = sp.Symbol("delta_yh")
# z displacement of the hook from the quadrotor center
delta_zh = sp.Symbol("delta_zh")
params = [m, I, g, l, mo, delta_yh, delta_zh]
action = [thrust, tau]


y, z, theta, phi = dynamicsymbols("y z theta phi")
y_dot, z_dot, theta_dot, phi_dot = sp.diff(y, t), sp.diff(
    z, t), sp.diff(theta, t), sp.diff(phi, t)
y_ddot, z_ddot, theta_ddot, phi_ddot = sp.diff(y_dot, t), sp.diff(
    z_dot, t), sp.diff(theta_dot, t), sp.diff(phi_dot, t)
y_dot_val, z_dot_val, theta_dot_val, phi_dot_val = sp.symbols(
    "y_dot_val z_dot_val theta_dot_val phi_dot_val")
y_ddot_val, z_ddot_val, theta_ddot_val, phi_ddot_val = sp.symbols(
    "y_ddot_val z_ddot_val theta_ddot_val phi_ddot_val")
states = [y, z, theta, phi, y_dot, z_dot, theta_dot, phi_dot]
states_val = [y, z, theta, phi, y_dot_val,
                z_dot_val, theta_dot_val, phi_dot_val]
states_dot = [y_ddot, z_ddot, theta_ddot, phi_ddot, f_rope]
states_dot_val = [y_ddot_val, z_ddot_val,
                    theta_ddot_val, phi_ddot_val, f_rope]

states_val = [y, z, theta, phi, y_dot_val,
                  z_dot_val, theta_dot_val, phi_dot_val]

# Solve for the acceleration
A_taut_dyn = sp.zeros(5, 5)
b_taut_dyn = sp.zeros(5, 1)

# lambda A_taut_dyn
A_taut_dyn_func = sp.lambdify(
    params + states_val + action, A_taut_dyn, "jax")
b_taut_dyn_func = sp.lambdify(
    params + states_val + action, b_taut_dyn, "jax")

In [1]:
from sympy import symbols
from sympy.physics.mechanics import dynamicsymbols, ReferenceFrame
from sympy.physics.mechanics import RigidBody, Particle, Point, outer
from sympy.physics.mechanics import linear_momentum, angular_momentum
from sympy.physics.vector import init_vprinting
init_vprinting(pretty_print=False)
m, M, l1 = symbols('m M l1')
q1d = dynamicsymbols('q1d')
N = ReferenceFrame('N')
O = Point('O')
O.set_vel(N, 0 * N.x)
Ac = O.locatenew('Ac', l1 * N.x)
P = Ac.locatenew('P', l1 * N.x)
a = ReferenceFrame('a')
a.set_ang_vel(N, q1d * N.z)
Ac.v2pt_theory(O, N, a), P.v2pt_theory(O, N, a)

(l1*q1d*N.y, 2*l1*q1d*N.y)

In [9]:
import sympy as sp
from sympy.physics.mechanics import (
    dynamicsymbols,
    ReferenceFrame,
    Point,
    Particle,
    RigidBody,
    inertia,
)
t = sp.Symbol("t")  # time
y, z, theta, phi = dynamicsymbols("y z theta phi")
y_dot, z_dot, theta_dot, phi_dot = sp.diff(y, t), sp.diff(
    z, t), sp.diff(theta, t), sp.diff(phi, t)
# y displacement of the hook from the quadrotor center
delta_yh = sp.Symbol("delta_yh")
# z displacement of the hook from the quadrotor center
delta_zh = sp.Symbol("delta_zh")

# Define inertial reference frame
N = ReferenceFrame("N")
N_origin = Point('N_origin')
A = N.orientnew("A", "Axis", [theta, N.x])
B = A.orientnew("B", "Axis", [phi, A.x])
# Define point
drone = Point("drone")
drone.set_pos(N_origin, y * N.y + z * N.z)
hook = drone.locatenew("hook", delta_yh * A.y + delta_zh * A.z)
drone, hook, N_origin, A.x, B.x


(drone, hook, N_origin, A.x, B.x)

In [7]:
import sympy as sp
from sympy.physics.mechanics import (
    dynamicsymbols,
    ReferenceFrame,
    Point,
    Particle,
    RigidBody,
    inertia,
)
# Define symbols
t = sp.Symbol("t")  # time
m = sp.Symbol("m", positive=True)  # mass of the quadrotor
I = sp.Symbol("I", positive=True)  # moment of inertia
g = sp.Symbol("g", positive=True)  # gravitational acceleration
l = sp.Symbol("l", positive=True)  # length of the rod
# mass of the object attached to the rod
mo = sp.Symbol("mo", positive=True)
thrust = sp.Function("thrust")(t)  # thrust force
tau = sp.Function("tau")(t)  # torque
f_rope = sp.Symbol("f_rope")  # force in the rope
thrust2 = sp.Function("thrust2")(t)  # thrust force
tau2 = sp.Function("tau2")(t)  # torque
f_rope2 = sp.Symbol("f_rope2")  # force in the rope
# y displacement of the hook from the quadrotor center
delta_yh = sp.Symbol("delta_yh")
# z displacement of the hook from the quadrotor center
delta_zh = sp.Symbol("delta_zh")
# y displacement of the hook from the quadrotor center
delta_yh2 = sp.Symbol("delta_yh2")
# z displacement of the hook from the quadrotor center
delta_zh2 = sp.Symbol("delta_zh2")
# TODO...
params = [m, I, g, l, mo, delta_yh, delta_zh]
action = [thrust, tau, thrust2, tau2]

# Define state variables and their derivatives
y, z, theta, phi = dynamicsymbols("y z theta phi")
y_dot, z_dot, theta_dot, phi_dot = sp.diff(y, t), sp.diff(
    z, t), sp.diff(theta, t), sp.diff(phi, t)
y_ddot, z_ddot, theta_ddot, phi_ddot = sp.diff(y_dot, t), sp.diff(
    z_dot, t), sp.diff(theta_dot, t), sp.diff(phi_dot, t)
y_dot_val, z_dot_val, theta_dot_val, phi_dot_val = sp.symbols(
    "y_dot_val z_dot_val theta_dot_val phi_dot_val")
y_ddot_val, z_ddot_val, theta_ddot_val, phi_ddot_val = sp.symbols(
    "y_ddot_val z_ddot_val theta_ddot_val phi_ddot_val")

y2, z2, theta2, phi2 = dynamicsymbols("y2 z2 theta2 phi2")
y2_dot, z2_dot, theta2_dot, phi2_dot = sp.diff(y2, t), sp.diff(
    z2, t), sp.diff(theta2, t), sp.diff(phi2, t)
y2_ddot, z2_ddot, theta2_ddot, phi2_ddot = sp.diff(y2_dot, t), sp.diff(
    z2_dot, t), sp.diff(theta2_dot, t), sp.diff(phi2_dot, t)
y2_dot_val, z2_dot_val, theta2_dot_val, phi2_dot_val = sp.symbols(
    "y2_dot_val z2_dot_val theta2_dot_val phi2_dot_val")
y2_ddot_val, z2_ddot_val, theta2_ddot_val, phi2_ddot_val = sp.symbols(
    "y2_ddot_val z2_ddot_val theta2_ddot_val phi2_ddot_val")

states = [y, z, theta, phi, y_dot, z_dot, theta_dot, phi_dot,
            y2, z2, theta2, phi2, y2_dot, z2_dot, theta2_dot, phi2_dot]
states_val = [y, z, theta, phi, y_dot_val,
                z_dot_val, theta_dot_val, phi_dot_val, 
                y2, z2, theta2, phi2, y2_dot_val, z2_dot_val, theta2_dot_val, phi2_dot_val]
states_dot = [y_ddot, z_ddot, theta_ddot, phi_ddot, f_rope, y2_ddot, z2_ddot, theta2_ddot, phi2_ddot, f_rope2]
states_dot_val = [y_ddot_val, z_ddot_val,
                    theta_ddot_val, phi_ddot_val, f_rope, 
                    y2_ddot_val, z2_ddot_val,
                    theta2_ddot_val, phi2_ddot_val, f_rope2]

# intermediate variables
delta_yh_global = delta_yh * sp.cos(theta) - delta_zh * sp.sin(theta)
delta_zh_global = delta_yh * sp.sin(theta) + delta_zh * sp.cos(theta)
f_rope_y = f_rope * sp.sin(theta+phi)
f_rope_z = -f_rope * sp.cos(theta+phi)
y_hook = y + delta_yh_global
z_hook = z + delta_zh_global
y_hook_dot = sp.diff(y_hook, t)
z_hook_dot = sp.diff(z_hook, t)
y_obj = y_hook + l * sp.sin(theta+phi)
z_obj = z_hook - l * sp.cos(theta+phi)
y_obj_dot = sp.diff(y_obj, t)
z_obj_dot = sp.diff(z_obj, t)
y_obj_ddot = sp.diff(y_obj_dot, t)
z_obj_ddot = sp.diff(z_obj_dot, t)
obses = [y_obj, z_obj, y_obj_dot, z_obj_dot,
            y_obj_ddot, z_obj_ddot, f_rope_y, f_rope_z]

delta_yh2_global = delta_yh2 * sp.cos(theta2) - delta_zh * sp.sin(theta2)
delta_zh2_global = delta_yh2 * sp.sin(theta2) + delta_zh * sp.cos(theta2)
f_rope2_y = f_rope2 * sp.sin(theta2+phi2)
f_rope2_z = -f_rope2 * sp.cos(theta2+phi2)
y_hook2 = y2 + delta_yh2_global
z_hook2 = z2 + delta_zh2_global
y_hook2_dot = sp.diff(y_hook2, t)
z_hook2_dot = sp.diff(z_hook2, t)
# obses = [y_obj, z_obj, y_obj_dot, z_obj_dot,
#          y_obj_ddot, z_obj_ddot, f_rope_y, f_rope_z]


# Define inertial reference frame
N = ReferenceFrame("N")
N_origin = Point('N_origin')
A = N.orientnew("A", "Axis", [theta, N.x])
B = A.orientnew("B", "Axis", [phi, A.x])

# Define point
drone = Point("drone")
drone.set_pos(N_origin, y * N.y + z * N.z)
hook = drone.locatenew("hook", delta_yh * A.y + delta_zh * A.z)
drone2 = Point("drone2")
drone2.set_pos(N_origin, y2 * N.y + z2 * N.z)
hook2 = drone2.locatenew("hook2", delta_yh2 * A.y + delta_zh2 * A.z)
obj = hook.locatenew("obj", -l * B.z)
drone.set_vel(N, y_dot * N.y + z_dot * N.z)
drone2.set_vel(N, y2_dot * N.y + z2_dot * N.z)

# Inertia
inertia_quadrotor = inertia(N, I, 0, 0)
quadrotor = RigidBody("quadrotor", drone, A, m, (inertia_quadrotor, drone))
quadrotor2 = RigidBody("quadrotor2", drone2, A, m, (inertia_quadrotor, drone2))
obj_particle = Particle("obj_particle", obj, mo)

# Newton's law
eq_quad_y = -thrust * sp.sin(theta) + f_rope_y - m * y_ddot
eq_quad_z = thrust * sp.cos(theta) + f_rope_z - m * g - m * z_ddot
eq_quad_theta = tau + delta_yh_global * f_rope_z - \
    delta_zh_global * f_rope_y - I * theta_ddot
# eq_obj_y = -f_rope_y - mo * y_obj_ddot
# eq_obj_z = -f_rope_z - mo * g - mo * z_obj_ddot

eq_quad_y2 = -thrust2 * sp.sin(theta2) + f_rope2_y - m * y2_ddot
eq_quad_z2 = thrust2 * sp.cos(theta2) + f_rope2_z - m * g - m * z2_ddot
eq_quad_theta2 = tau2 + delta_yh2_global * f_rope2_z - \
    delta_zh2_global * f_rope2_y - I * theta2_ddot
eq_obj_y = -f_rope_y -f_rope2_y - mo * y_obj_ddot
eq_obj_z = -f_rope_z -f_rope2_z - mo * g - mo * z_obj_ddot


eqs = [eq_quad_y, eq_quad_z, eq_quad_theta, eq_obj_y, eq_obj_z, eq_quad_y2, eq_quad_z2, eq_quad_theta2]
eqs = [eq.expand() for eq in eqs]
eqs = [eq.subs([(states_dot[i], states_dot_val[i])
                for i in range(len(states_dot))]) for eq in eqs]
eqs = [eq.subs([(states[i], states_val[i])
                for i in range(len(states))]) for eq in eqs]
# Solve for the acceleration
A_taut_dyn = sp.zeros(5, 5)
b_taut_dyn = sp.zeros(5, 1)
for i in range(5):
    for j in range(5):
        A_taut_dyn[i, j] = eqs[i].coeff(states_dot_val[j])
    b_taut_dyn[i] = -eqs[i].subs([(states_dot_val[j], 0)
                                    for j in range(5)])
# lambda A_taut_dyn
A_taut_dyn_func = sp.lambdify(
    params + states_val + action, A_taut_dyn, "jax")
b_taut_dyn_func = sp.lambdify(
    params + states_val + action, b_taut_dyn, "jax")
A_taut_dyn_func, b_taut_dyn_func

(<function _lambdifygenerated(m, I, g, l, mo, delta_yh, delta_zh, _Dummy_173, _Dummy_171, _Dummy_177, _Dummy_181, y_dot_val, z_dot_val, theta_dot_val, phi_dot_val, _Dummy_172, _Dummy_170, _Dummy_176, _Dummy_180, y2_dot_val, z2_dot_val, theta2_dot_val, phi2_dot_val, _Dummy_175, _Dummy_179, _Dummy_174, _Dummy_178)>,
 <function _lambdifygenerated(m, I, g, l, mo, delta_yh, delta_zh, _Dummy_185, _Dummy_183, _Dummy_189, _Dummy_193, y_dot_val, z_dot_val, theta_dot_val, phi_dot_val, _Dummy_184, _Dummy_182, _Dummy_188, _Dummy_192, y2_dot_val, z2_dot_val, theta2_dot_val, phi2_dot_val, _Dummy_187, _Dummy_191, _Dummy_186, _Dummy_190)>)

In [17]:

# Solve for equation
obs_eqs = [y_obj, z_obj, y_obj_dot, z_obj_dot, y_hook,
            z_hook, y_hook_dot, z_hook_dot, y_hook2, z_hook2, f_rope_y, f_rope_z, f_rope2_y, f_rope2_z]
# replace states with states_val, states_dot with states_dot_val
obs_eqs = [eq.subs([(states_dot[i], states_dot_val[i])
                    for i in range(len(states_dot))]) for eq in obs_eqs]
obs_eqs = [eq.subs([(states[i], states_val[i])
                    for i in range(len(states))]) for eq in obs_eqs]

# lambda obs_eqs
obs_eqs_func = sp.lambdify(
    params + states_val + states_dot_val + action, obs_eqs, "jax")
# lambda some other obs functions
y_hook_func = sp.lambdify(params + states_val, obs_eqs[4], "jax")
z_hook_func = sp.lambdify(params + states_val, obs_eqs[5], "jax")
y_hook_dot_func = sp.lambdify(params + states_val, obs_eqs[6], "jax")
z_hook_dot_func = sp.lambdify(params + states_val, obs_eqs[7], "jax")


obs_eqs
# obs_eqs_func
import sys
sys.path.append("/home/bianzx/jax/quadjax/")
from jax import numpy as jnp
from typing import Tuple
from quadjax.dynamics.dataclass import EnvParams, EnvState, Action
from quadjax.dynamics.utils import angle_normalize

def taut_dynamics(env_params: EnvParams, env_state: EnvState, env_action: Tuple[Action, Action]):
        params = [env_params.m, env_params.I, env_params.g, env_params.l,
                  env_params.mo, env_params.delta_yh, env_params.delta_zh]
        states = [env_state.y, env_state.z, env_state.theta, env_state.phi,
                  env_state.y_dot, env_state.z_dot, env_state.theta_dot, env_state.phi_dot,
                  env_state.y2, env_state.z2, env_state.theta2, env_state.phi2,
                  env_state.y2_dot, env_state.z2_dot, env_state.theta2_dot, env_state.phi2_dot]
        # action = [env_action.thrust, env_action.tau]
        action = [env_action[0].thrust, env_action[0].tau,
                env_action[1].thrust, env_action[1].tau]
        A = A_taut_dyn_func(*params, *states, *action)
        b = b_taut_dyn_func(*params, *states, *action)
        states_dot = jnp.linalg.solve(A, b).squeeze()
        # y_ddot, z_ddot, theta_ddot, phi_ddot, f_rope = states_dot
        y_ddot, z_ddot, theta_ddot, phi_ddot, f_rope, y2_ddot, z2_ddot, theta2_ddot, phi2_ddot, f_rope2 = states_dot

        # Calculate updated state variables
        new_y_dot = env_state.y_dot + y_ddot * env_params.dt
        new_z_dot = env_state.z_dot + z_ddot * env_params.dt
        new_theta_dot = env_state.theta_dot + theta_ddot * env_params.dt
        new_phi_dot = env_state.phi_dot + phi_ddot * env_params.dt
        new_y = env_state.y + new_y_dot * env_params.dt
        new_z = env_state.z + new_z_dot * env_params.dt
        new_theta = angle_normalize(
            env_state.theta + new_theta_dot * env_params.dt)
        new_phi = angle_normalize(env_state.phi + new_phi_dot * env_params.dt)
        
        new_y2_dot = env_state.y2_dot + y2_ddot * env_params.dt
        new_z2_dot = env_state.z2_dot + z2_ddot * env_params.dt
        new_theta2_dot = env_state.theta2_dot + theta2_ddot * env_params.dt
        new_phi2_dot = env_state.phi2_dot + phi2_ddot * env_params.dt
        new_y2 = env_state.y2 + new_y2_dot * env_params.dt
        new_z2 = env_state.z2 + new_z2_dot * env_params.dt
        new_theta2 = angle_normalize(
            env_state.theta2 + new_theta2_dot * env_params.dt)
        new_phi2 = angle_normalize(env_state.phi2 + new_phi2_dot * env_params.dt)

        # Update states list
        states = [new_y, new_z, new_theta, new_phi,
                  new_y_dot, new_z_dot, new_theta_dot, new_phi_dot,
                  new_y2, new_z2, new_theta2, new_phi2,
                  new_y2_dot, new_z2_dot, new_theta2_dot, new_phi2_dot]

        # Compute other state variables
        y_obj, z_obj, y_obj_dot, z_obj_dot, y_hook, z_hook, y_hook_dot, z_hook_dot, f_rope_y, f_rope_z, y_hook2, z_hook2, y_hook2_dot, z_hook2_dot, f_rope2_y, f_rope2_z = obs_eqs_func(
            *params, *states, *states_dot, *action)

        # Update all state variables at once using replace()
        env_state = env_state.replace(
            y=new_y,
            z=new_z,
            theta=new_theta,
            phi=new_phi,
            y_dot=new_y_dot,
            z_dot=new_z_dot,
            theta_dot=new_theta_dot,
            phi_dot=new_phi_dot,
            y_obj=y_obj,
            z_obj=z_obj,
            y_obj_dot=y_obj_dot,
            z_obj_dot=z_obj_dot,
            y_hook=y_hook,
            z_hook=z_hook,
            y_hook_dot=y_hook_dot,
            z_hook_dot=z_hook_dot,
            y2=new_y2,
            z2=new_z2,
            theta2=new_theta2,
            phi2=new_phi2,
            y2_dot=new_y2_dot,
            z2_dot=new_z2_dot,
            theta2_dot=new_theta2_dot,
            phi2_dot=new_phi2_dot,
            y_hook2=y_hook2,
            z_hook2=z_hook2,
            y_hook2_dot=y_hook2_dot,
            z_hook2_dot=z_hook2_dot,
            f_rope_y=f_rope_y,
            f_rope_z=f_rope_z,
            f_rope=f_rope,
            l_rope=env_params.l,
            last_thrust=env_action[0].thrust,
            last_tau=env_action[0].tau,
            f_rope2_y=f_rope2_y,
            f_rope2_z=f_rope2_z,
            f_rope2=f_rope2,
            l_rope2=env_params.l,
            last_thrust2=env_action[1].thrust,
            last_tau2=env_action[1].tau,
            time=env_state.time + 1,
            y_tar=env_state.y_traj[env_state.time],
            z_tar=env_state.z_traj[env_state.time],
            y_dot_tar=env_state.y_dot_traj[env_state.time],
            z_dot_tar=env_state.z_dot_traj[env_state.time],
        )

        return env_state

y_obj, z_obj, y_obj_dot, z_obj_dot, y_hook, z_hook, y_hook_dot, z_hook_dot, f_rope_y, f_rope_z, y_hook2, z_hook2, y_hook2_dot, z_hook2_dot, f_rope2_y, f_rope2_z

SyntaxError: keyword argument repeated: l_rope (303233973.py, line 77)